# LES 1 – Klimaatdata begrijpen en verkennen

## 1. Wat bekijken we?

We werken met een echte dataset over klimaatverandering.

<iframe src="https://ourworldindata.org/grapher/temperature-anomaly?time=earliest..2025&tab=line" loading="lazy" style="width: 100%; height: 600px; border: 0px none;" allow="web-share; clipboard-write"></iframe>

De gegevens komen van een internationaal klimaatonderzoek (HadCRUT5) en worden gepubliceerd via Our World in Data. Het gaat om:

> **Temperatuuranomalieën ten opzichte van 1861–1890**

Belangrijk:
Dit zijn **geen absolute temperaturen**.

Een waarde van **+1.2 °C** betekent:

$
\text{Temperatuur in dat jaar} - \text{gemiddelde temperatuur 1861–1890}
$

Met andere woorden:

Als het gemiddelde tussen 1861 en 1890 bijvoorbeeld 14.0 °C was,
en een bepaald jaar gemiddeld 15.2 °C was,
dan is de anomalie:

$
15.2 - 14.0 = 1.2^\circ C
$

We werken dus met **verschillen**, niet met echte temperaturen. Dat maakt trends duidelijker zichtbaar.

### Upper bound en Lower bound

In de dataset staan drie waarden per jaar:

* **Average**
* **Lower bound**
* **Upper bound**

Volgens de metadata  zijn de laatste twee:

> de onder- en bovengrens van het **95% betrouwbaarheidsinterval** van de jaarlijkse temperatuuranomalie.

#### Wat betekent dat concreet?

De gemeten temperatuur is geen exacte waarde.
Er is altijd onzekerheid, omdat:

* metingen vroeger schaars waren
* niet elke plek op aarde gemeten werd
* meetmethodes veranderden
* sommige regio’s ontbreken in bepaalde jaren

Daarom geeft men niet alleen een centrale schatting (Average), maar ook een onzekerheidsinterval.

Stel dat voor 1865 de anomalie is:

Average = 0.42 °C

Dan kan het interval bijvoorbeeld zijn:

* Lower bound = 0.35 °C
* Upper bound = 0.49 °C

Dat betekent:

> Met 95% zekerheid ligt de echte waarde tussen 0.35 en 0.49 °C.

Niet:

* dat het 95% waarschijnlijk is dat het exact 0.42 is
* maar dat 0.42 de beste schatting is

> In deze les werken we rechtstreeks met deze data. Je kan die downloaden via de download functie bij deze grafiek. Kies dan voor `Data` -> `download full data`. Je download een zip bestand dat je zelf moet uitpakken.

## 2. De CSV openen in Excel

Voor we Python gebruiken, bekijken we eerst de ruwe data.

Een csv bestand kan je standaard openen met excel. Maar als je dat doet staan alle gegevens in de eerste kolom. Dat is dus niet de beste manier.

### Open het bestand correct:

1. Open Excel
2. Ga naar **Gegevens**
3. Kies **Uit tekst/CSV**
4. Selecteer het bestand
5. Controleer of Excel het scheidingsteken correct herkent (komma)

Een CSV-bestand is een **tekstbestand is waarin waarden gescheiden worden door komma’s**.

CSV = *Comma Separated Values*

### Wat zien we?

Bekijk de kolommen.

Typische kolommen in deze dataset:

* **Entity** → regio (World, Northern Hemisphere, Southern Hemisphere)
* **Code** → laat je weten waar de data vandaan komt (niet belangrijk voor ons)
* **Year** → jaartal
* **Average** → gemiddelde temperatuur van een jaar
* lower bound / upper bound (betrouwbaarheidsinterval)

Let op:

Je ziet soms “grote” getallen zoals:

```
-55067
1615803
14512725
```
Dit gebeurt als de taalinstellingen op je laptop op 'België' staan. Bij internationale data wordt er een punt gebruikt om decimalen aan te duiden. Vandaar dat we spreaking over `floating point` numbers. Hierdoor zal Excel je data fout interpreteren.

### CSV in VSCode

Omdat Excel minder geschikt is voor deze data, kan je je bestand beter openen in VSCode. Als je dan ook nog de extensie `Rainbow CSV` installeert, dan krijg je een beter beeld van je data.

> Als je rechts klikt in je csv bestand, dan krijg je ook een menu `Rainbow CSV`. Als je daar mee experimenteert kan je je data nog duidelijker weergeven.

### Vragen

1. Welke jaren bevat de dataset?
2. Hoeveel verschillende regio’s zie je?
3. Wat valt je op als je naar de recente jaren kijkt?
4. Zie je negatieve waarden? Wat betekenen die?

---

## 3. Inlezen en verkennen met pandas

Nu gebruiken we Python om de dataset systematisch te analyseren. Daarvoor hebben we opnieuw een library nodig: `pandas`. Je kan die installeren met `pip install pandas`.

### Stap 1 – Inlezen

```python
import pandas as pd

# pandas noemt een set met gegevens een 'dataframe'
# daarom geven we deze variabele de naam 'df'
df = pd.read_csv("temperature-anomaly.csv")
```
> Hier is het belangrijk dat het csv bestand in dezelfde map staat als waar je de python script uitvoert. Als dat niet zo is, dan zal python script vastlopen. Je kan het script dan in de console stoppen via `CTRL+C`.

---

### Stap 2 – Eerste inspectie

Voer de volgende commando’s uit:

```python
result = df.head()
print(result)
```

Wat toont dit?

```python
result = df.shape
print(result)
```

Wat betekent het eerste getal?
Wat betekent het tweede getal?

```python
result = df.dtypes
print(result)
```

Zijn alle kolommen van het juiste type? Zoek op wat `str`, `int` en `float` betekenen als je niet zeker bent.

---

### Stap 3 – Filteren op regio

We willen voorlopig enkel werken met de **Northern Hemisphere**. Die identifier vinden we in de kolom 'Entity'. Als we een filter maken, dan kunnen we enkel die gegevens tonen. 

```python
filter = df["Entity"] == "Northern Hemisphere"
north = df[filter]

result = north.head()
print(result)
```

Je kan dat trouwens ook in één regel doen. De versie hierboven is misschien wel duidelijker, maar als je code van anderen leest, dan zal je eerder dit tegenkomen:

```python
print(df[df["Entity"] == "Northern Hemisphere"].head())
```

Controleer:

```python
print(df["Entity"].unique())

# en daarna:
north = df[df["Entity"] == "Northern Hemisphere"]
print(north["Entity"].unique())
```

Wat krijg je? Waaneer is het zinvol om eerst één regio te selecteren?

---

### Opdracht

Beantwoord:

1. Hoeveel rijen bevat de Northern Hemisphere?
2. Wat is het eerste jaartal?
3. Wat is het laatste jaartal?

Hint: je kan experimenteren met de functies van pandas:

```python
north.info()
north["Year"].???() # welke functies zouden er kunnen bestaan om het kleinstse of grootste getal te vinden?
```


# 4. Controleren van de referentieperiode (baseline)

We weten dat deze dataset werkt met een referentieperiode:

> 1861–1890 = gemiddelde anomalie ≈ 0 °C

Dat betekent: De anomalieën zijn zo berekend dat het gemiddelde van die periode nul is.

We gaan dit controleren.


## 4.1 Filteren op de referentieperiode

We werken voorlopig met de gefilterde dataset `north` (Northern Hemisphere).

Selecteer de jaren 1861 tot en met 1890:

```python
baseline = north[(north["Year"] >= 1861) & (north["Year"] <= 1890)]
```

Controleer hoeveel rijen je krijgt door de pandas functies uit de vorige stap te gebruiken.

## 4.2 Gemiddelde berekenen

Bereken het gemiddelde van de anomalie in deze periode.

Gebruik de kolomnaam van de gemiddelde anomalie (controleer exact hoe ze in jouw dataset heet).

> Je weet nog niet welke functie je nodig hebt om het gemiddelde op te vragen. Wat zou de naam van die functie zijn?

## 4.3 Interpretatie

Waarschijnlijk krijg je een waarde zeer dicht bij 0, maar niet exact 0.

Waarom?

* Afrondingsfouten
* Statistische verwerking
* Onzekerheid in metingen

Belangrijk inzicht:

> De anomalieën zijn zo geschaald dat het gemiddelde van 1861–1890 ongeveer nul is.

Dit is een controle op de betrouwbaarheid van deze dataset.

# 5. Eerste grafiek met matplotlib

We willen nu de evolutie van de anomalie doorheen de tijd visualiseren.

Vraag eerst aan jezelf:

* Wat komt op de x-as?
* Wat komt op de y-as?

## 5.1 Data selecteren

We halen eerst de juiste kolommen uit de dataset.

```python
years = north["?"] # kies je kolom
anomaly = north["?"] # kies je kolom
```

Controleer wat je nu hebt:

```python
years.head() # toon dit op je scherm
anomaly.head() # toon dit op je scherm
```

## 5.2 Grafiek maken

We gebruiken nu opnieuw matplotlib (die je in eerdere modules al gebruikte).

```python
import matplotlib.pyplot as plt

plt.figure()
plt.plot(years, anomaly)
plt.xlabel("Year")
plt.ylabel("Temperature anomaly (°C)")
plt.title("Northern Hemisphere temperature anomaly")
plt.grid(True)
plt.show()
```

---

## 5.3 Analyse-opdracht

1. Vanaf wanneer zie je een duidelijke stijging?
2. Zijn er periodes waarin de temperatuur tijdelijk daalt?
3. Is de stijging lineair?
4. Wat valt je op na 1980?

---

## 5.4 Oefeningen

* Kan je in deze grafiek ook de Lower en upper bounds tonen?
* Hoe zoom je in op de laatste 50 jaar?
* Toon het noordelijk en zuidelijk halfrond in één grafiek.

### Oefeningen met AI
Bij de volgende oefeningen is het de bedoeling dat je AI gebruikt om je te helpen. Je krijgt telkens het antwoord op de vraag. Gebruik dit om te controleren of de code die je via AI krijgt, klopt.

* In welk jaar was de grootste stijging t.o.v. het vorige jaar? (1863)
* In welk jaar werd voor het eerst 1°C overschreden? (1998)

* Toon een grafiek waarin je de waarden groepeert per 10 jaar.
![image.png](/figures/module04/decades.png)

* Toon een grafiek waarin je het voortschrijdend gemiddelde ziet (over 10 jaar).
![image.png](/figures/module04/rolling_average.png)

* En nu zonder AI: hoe pas je de vorige grafiek aan zodat je een voortschrijdend gemiddelde over 20 jaar ziet?






# 6. Afsluiting – Wat is hier écht belangrijk?

In deze les hebben we gewerkt met een echte klimaatdataset. We hebben:

* een CSV-bestand correct geïmporteerd
* begrepen wat een temperatuuranomalie is
* gecontroleerd dat de referentieperiode (1861–1890) gemiddeld rond 0 ligt
* data gefilterd op één regio
* een grafiek gemaakt
* onzekerheidsgrenzen geïnterpreteerd

Maar het belangrijkste van deze les is niet de exacte Python-commando’s.

Wat je moet onthouden:

## 6.1 Begrijp wat je data voorstelt

Voor je iets berekent of tekent, moet je weten:

* Wat betekent deze kolom?
* Wat is de eenheid?
* Wat is de referentie?
* Zijn dit absolute waarden of verschillen?

Zonder inhoudelijk begrip is een grafiek betekenisloos.

## 6.2 Controleer vóór je analyseert

Altijd eerst:

* `head()` bekijken
* `shape` controleren
* nagaan of je juiste subset gebruikt
* logisch nadenken over wat je verwacht

Een goede analyse begint met inspectie.


## 6.3 Grafieken zijn denktools

Een grafiek is geen versiering.

Ze helpt je:

* trends te zien
* fouten te ontdekken
* redeneringen te controleren

Visualisatie is een vorm van denken.

## 6.4 Je moet geen pandas vanbuiten kennen

Het is niet de bedoeling dat je:

* alle functies uit het hoofd kent
* exacte syntaxis memoriseert

Wat wél belangrijk is:

* begrijpen wat je doet
* weten welk soort bewerking je nodig hebt
* documentatie kunnen opzoeken
* fouten herkennen

Programmeren in een wetenschappelijke context betekent:

> weten wat je wil berekenen
> begrijpen wat het resultaat betekent
> en kunnen controleren of het logisch is.
